#cd C:\kafka
#zookeeper-server-start.bat C:\kafka\config\zookeeper.properties
#kafka-server-start.bat C:\kafka\config\server.properties

In [10]:
from kafka import KafkaProducer
from time import sleep
from json import dumps
import urllib.request
import pandas as pd

In [11]:
df = pd.read_csv('load_archive.csv')

In [13]:
df.shape[0]

11830

In [5]:
type(df.iloc[0].to_dict())

dict

In [3]:
external_ip = urllib.request.urlopen('https://ident.me').read().decode('utf8')
external_ip

'114.109.208.141'

In [4]:
import logging
logging.basicConfig(level=logging.DEBUG)
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8'))

DEBUG:kafka.producer.kafka:Starting the Kafka producer
DEBUG:kafka.metrics.metrics:Added sensor with name connections-closed
DEBUG:kafka.metrics.metrics:Added sensor with name connections-created
DEBUG:kafka.metrics.metrics:Added sensor with name select-time
DEBUG:kafka.metrics.metrics:Added sensor with name io-time
DEBUG:kafka.client:Initiating connection to node bootstrap-0 at localhost:9092
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent-received
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name request-latency
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.latency
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <disconnected> [unspecif

In [ ]:
sample_data = df.iloc[0].to_dict()
producer.send('load', value=sample_data)

In [5]:
producer.flush()

In [6]:
producer.close()

DEBUG:kafka.producer.sender:Beginning shutdown of Kafka producer I/O thread, sending remaining records.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Closing connection. 
DEBUG:kafka.producer.sender:Shutdown of Kafka producer I/O thread has completed.


In [2]:
print(pd.Timestamp.now())

2023-12-25 17:12:46.711187


#consumer

In [7]:
from kafka import KafkaConsumer
from json import loads, dumps


try:
    consumer = KafkaConsumer('Ptot',
                             bootstrap_servers=['localhost:9092'],
                             value_deserializer=lambda x: loads(x.decode('utf-8')),
                             request_timeout_ms = 10000)
except Exception as e:
    print("An error occurred while initializing the Kafka consumer:", e)
    consumer = None

DEBUG:kafka.metrics.metrics:Added sensor with name connections-closed
DEBUG:kafka.metrics.metrics:Added sensor with name connections-created
DEBUG:kafka.metrics.metrics:Added sensor with name select-time
DEBUG:kafka.metrics.metrics:Added sensor with name io-time
DEBUG:kafka.client:Initiating connection to node bootstrap-0 at localhost:9092
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent-received
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name request-latency
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.latency
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <disconnected> [unspecified None]>: creating new socket
DEBUG:kafka.conn:<Broke

In [ ]:
for message in consumer:
    
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    # print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
    #                                       message.offset, message.key,
    #                                       message.value))
    print(pd.DataFrame.from_dict(message.value,orient='index'))

In [9]:
consumer.close()

DEBUG:kafka.consumer.group:Closing the KafkaConsumer.
INFO:kafka.conn:<BrokerConnection node_id=0 host=host.docker.internal:9092 <connected> [IPv4 ('192.168.1.39', 9092)]>: Closing connection. 
DEBUG:kafka.conn:<BrokerConnection node_id=0 host=host.docker.internal:9092 <connected> [IPv4 ('192.168.1.39', 9092)]>: reconnect backoff 0.04444703085128445 after 1 failures
ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: Cancelled: <BrokerConnection node_id=0 host=host.docker.internal:9092 <connected> [IPv4 ('192.168.1.39', 9092)]>
DEBUG:kafka.consumer.group:The KafkaConsumer has closed.


In [7]:
d = {'datetime': '1/3/2023 12:00:00', 'Ia(A)': 13.92311756, 'Ib(A)': 13.33506956, 'Ic(A)': 11.69988856, 'In(A)': 0.0, 'Iavg(A)': 0.0, 'Va(V)': 0.0, 'Vb(V)': 0.0, 'Vc(V)': 0.0, 'Vavg(V)': 0.0, 'Pa(kW)': 1.442908766, 'Pb(kW)': 3.007359379, 'Pc(kW)': 2.865918476, 'Ptot(kW)': 7.313675841, 'VARa(kVAR)': 0.0, 'VARb(kVAR)': 0.0, 'VARc(kVAR)': 0.0, 'VARtot(kVAR)': 0.0, 'VAa(V)': 0.0, 'VAb(V)': 0.0, 'VAc(V)': 0.0, 'VAtot(V)': 0.0, 'PFa': 0.0, 'PFb': 0.0, 'PFc': 0.0, 'PFtot': 0.0, 'DFa': 0.0, 'DFb': 0.0, 'DFc': 0.0, 'DFtot': 0.0}

In [10]:
data = pd.DataFrame.from_dict(d,orient='index').T

In [11]:
data['datetime']

0    1/3/2023 12:00:00
Name: datetime, dtype: object